# Unit Commitment

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.5.post85.dev0+gfd25f07'

In [3]:
ams.config_logger(stream_level=10)

In [4]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.2201 seconds.
Adjust bus index to start from 0.
System set up in 0.0095 seconds.


In [5]:
sp.RTED.run()

Setup model of RTED
RTED data check passed.
- Generating symbols for RTED
Set constrs pb: sum(pd1) + sum(pd2) - sum(pg)
Set constrs lub: PTDF1 @ (pg - pd1) - PTDF2 * pd2 - rate_a
Set constrs llb: - PTDF1 @ (pg - pd1) + PTDF2 * pd2 - rate_a
Set constrs rbu: gs @ pru - du
Set constrs rbd: gs @ prd - dd
Set constrs rru: pg + pru - pmax
Set constrs rrd: -pg + prd - pmin
Set constrs rgu: pg - pg0 - R10
Set constrs rgd: -pg + pg0 - R10
RTED model set up in 0.0518 seconds.
RTED solved as optimal in 0.0238 seconds with exit code 0.


True

In [6]:
sp = ams.load(ams.get_case('ieee14/ieee14_uced.xlsx'),
              setup=True,
              )

Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_uced.xlsx"...
Input file parsed in 0.1006 seconds.
Adjust bus index to start from 0.
System set up in 0.0186 seconds.


In [7]:
sp.ED.setup(show_code=True)

ED data check passed.
- Generating symbols for ED
Set constrs pb: pds - Spg @ pg
Code Constr: om.constrs["pb"]=self.om.rtn.pds.v - self.om.rtn.Spg.v @ self.om.pg == 0
Set constrs lub: PTDF1@pg - PTDF1@pd1s - PTDF2@pd2s - RAr@cdup
Code Constr: om.constrs["lub"]=self.om.rtn.PTDF1.v@self.om.pg - self.om.rtn.PTDF1.v@self.om.rtn.pd1s.v - self.om.rtn.PTDF2.v@self.om.rtn.pd2s.v - self.om.rtn.RAr.v@self.om.rtn.cdup.v <= 0
Set constrs llb: -PTDF1@pg + PTDF1@pd1s + PTDF2@pd2s - RAr@cdup
Code Constr: om.constrs["llb"]=-self.om.rtn.PTDF1.v@self.om.pg + self.om.rtn.PTDF1.v@self.om.rtn.pd1s.v + self.om.rtn.PTDF2.v@self.om.rtn.pd2s.v - self.om.rtn.RAr.v@self.om.rtn.cdup.v <= 0
Set constrs rgu: pg @ Mr - RR30
Code Constr: om.constrs["rgu"]=self.om.pg @ self.om.rtn.Mr.v - self.om.rtn.RR30.v <= 0
Set constrs rgd: -pg @ Mr - RR30
Code Constr: om.constrs["rgd"]=-self.om.pg @ self.om.rtn.Mr.v - self.om.rtn.RR30.v <= 0
Code Obj: om.obj=cp.Minimize(cp.sum(self.om.rtn.c2.v @ self.om.pg**2 + self.om.rtn.c1.v @

True

In [8]:
sp.ED.run()

ED solved as optimal in 0.0264 seconds with exit code 0.


True